In [1]:
!pip install datasets
!pip install pickel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
ERROR: Could not find a version that satisfies the requirement p

In [2]:
import jax
import jax.numpy as jnp
from jax import grad, jit, random
from jax import lax
from jax.tree_util import tree_map
import numpy as np
from jax import grad, jit, random
from jax import lax
from jax.tree_util import tree_map
from transformers import GPT2Model, GPT2Tokenizer
import torch
import numpy as np
from datasets import load_dataset

In [3]:
# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model_gpt = GPT2Model.from_pretrained('gpt2')
vocab = tokenizer.get_vocab()
model_gpt.eval()

def break_word_gpt(text):
    tokens = tokenizer.tokenize(text)
    return tokens

def break_into_id(text):
    return tokenizer.encode(text, add_special_tokens=False)

def encode_word_gpt(word):
    token_ids = tokenizer.encode(word, add_special_tokens=False)
    token_tensor = torch.tensor([token_ids])
    with torch.no_grad():
        token_embeddings = model_gpt.wte(token_tensor)
    return token_embeddings

# Load the PTB text dataset
# dataset = load_dataset("wikitext", "wikitext-103-raw-v1")
dataset = load_dataset('ptb_text_only')
train_data = dataset['train']
test_data = dataset['test']
print(dataset)

def read_batch_file(start_index, size, data):
    data_token = []
    for sentence_pair_index in range(start_index, start_index + size):
        if sentence_pair_index < len(data):
          if len(data[sentence_pair_index]["sentence"]) > 0:
              data_token.append(break_word_gpt(data[sentence_pair_index]["sentence"]))
    return data_token
def make_data(no_token,data,factor):
  data_tokens = read_batch_file(0, int(len(data)/factor), data)
  encode_tokens , encode_tokens_test ,x , y, y_LLM = [],[],[],[],[]
  temp = np.zeros(2*768)
  for i in data_tokens:
      encode_tokens.append(encode_word_gpt(i)[0])
  for i in range(len(data_tokens)):
      if len(data_tokens[i]) > no_token:
          y.append(encode_tokens[i][no_token])
          y_LLM.append(break_into_id([data_tokens[i][no_token]]))
  x = np.zeros(shape=(len(y),no_token // 2,2*768))
  count_index = 0
  for i in range(len(data_tokens)):
      if len(data_tokens[i]) > no_token:
          for k in range(no_token):
              if k % 2 == 0:
                  x[count_index][int(k/2)][:768] = encode_tokens[i][k]
              else:
                  x[count_index][int(k/2)][768:] = encode_tokens[i][k]
          count_index += 1
  x = np.transpose(np.transpose(x))
  y = jnp.array(y)
  x = jnp.array(x)
  return (x, y, y_LLM)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

The repository for ptb_text_only contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ptb_text_only.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/42068 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3761 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3370 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 42068
    })
    test: Dataset({
        features: ['sentence'],
        num_rows: 3761
    })
    validation: Dataset({
        features: ['sentence'],
        num_rows: 3370
    })
})


In [4]:
x, y, LLM_y = make_data(4, train_data,1)
x_test, y_test, LLM_y_test = make_data(4, test_data,1)
print(x.shape,x_test.shape)
print(y.shape,y_test.shape)

(41358, 2, 1536) (3692, 2, 1536)
(41358, 768) (3692, 768)


In [6]:
# Initialize network parameters
def init_network_params(layer_sizes, key):
    keys = random.split(key, len(layer_sizes))
    return [(random.normal(k, (m, n)) * 0.01, random.normal(k, (n,)) * 0.01)
            for m, n, k in zip(layer_sizes[:-1], layer_sizes[1:], keys)]

# Initialize Adam optimizer state
def init_adam_state(params):
    return [(jnp.zeros_like(w), jnp.zeros_like(w), jnp.zeros_like(b), jnp.zeros_like(b)) for w, b in params]

# Adam optimizer update
def adam_update(params, grads, opt_state, t, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
    new_params = []
    new_opt_state = []

    for (w, b), (dw, db), (m_w, v_w, m_b, v_b) in zip(params, grads, opt_state):
        m_w = beta1 * m_w + (1 - beta1) * dw
        v_w = beta2 * v_w + (1 - beta2) * (dw ** 2)
        m_b = beta1 * m_b + (1 - beta1) * db
        v_b = beta2 * v_b + (1 - beta2) * (db ** 2)

        m_w_hat = m_w / (1 - beta1 ** t)
        v_w_hat = v_w / (1 - beta2 ** t)
        m_b_hat = m_b / (1 - beta1 ** t)
        v_b_hat = v_b / (1 - beta2 ** t)

        w = w - lr * m_w_hat / (jnp.sqrt(v_w_hat) + eps)
        b = b - lr * m_b_hat / (jnp.sqrt(v_b_hat) + eps)

        new_params.append((w, b))
        new_opt_state.append((m_w, v_w, m_b, v_b))

    return new_params, new_opt_state

# Activation functions
def relu(x):
    return jnp.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + jnp.exp(-x))

def tanh(x):
    return jnp.tanh(x)

def softmax(x):
    exps = jnp.exp(x - jnp.max(x))
    return exps / exps.sum(axis=-1, keepdims=True)

# Define the neural network forward pass with activation function indices
def forward_pass(params, x, activation_indices):
    activations_output = x
    for i, (w, b) in enumerate(params[:-1]):
        outputs = jnp.dot(activations_output, w) + b
        activations_output = apply_activation(outputs, activation_indices[i])
    final_w, final_b = params[-1]
    return apply_activation(jnp.dot(activations_output, final_w) + final_b,activation_indices[-1])
    # return jnp.dot(activations_output, final_w) + final_b

# Helper function to apply activation functions using lax.cond
def apply_activation(x, activation_index):
    return lax.switch(activation_index, [relu, sigmoid, tanh, softmax], x)

# Loss function: Mean Squared Error
def process_tokens_single_example(params1, params2, params3, x, activation_indices1, activation_indices2, activation_indices3):
  for layer_idx in range(int(np.log2(x.shape[0]))):
    if layer_idx == 0:
      temp = forward_pass(params1, x, activation_indices1)
    else:
      temp = forward_pass(params1, input_next_layer, activation_indices1)
    input_next_layer = jnp.zeros(shape = (temp.shape[0] // 2 , 2*768))
    for k in range(temp.shape[0]):
      if k % 2 == 0:
          input_next_layer = input_next_layer.at[k // 2,:768].set(temp[k])
      else:
        input_next_layer = input_next_layer.at[k // 2 ,768:].set(temp[k])
  EO = forward_pass(params1, input_next_layer, activation_indices1)
  return forward_pass(params2, EO, activation_indices2)
def loss_single_example(params1, params2, params3, x, y, activation_indices1, activation_indices2, activation_indices3):
    next_predicted_token_embedding = process_tokens_single_example(params1, params2, params3, x, activation_indices1, activation_indices2, activation_indices3)
    return jnp.sum(jnp.abs(next_predicted_token_embedding - y))
loss_vectorized = jax.vmap(fun = loss_single_example, in_axes=(None,None,None,0,0,None,None,None))
process_tokens_vectorized = jax.vmap(fun = process_tokens_single_example, in_axes=(None, None, None, 0, None, None, None))
def learning_rate_decay(initial_lr, epoch, decay_rate=0.1, decay_steps=10):
    return initial_lr * (decay_rate ** (epoch // decay_steps))

def loss(params1, params2, params3, x, y, activation_indices1, activation_indices2, activation_indices3):
    return jnp.sum(loss_vectorized(params1, params2, params3, x, y, activation_indices1, activation_indices2, activation_indices3))
# Training step
@jit
def update(params1, params2, params3, opt_state1, opt_state2, opt_state3, x, y, activation_indices1, activation_indices2, activation_indices3, t, lr):
  batch_size = 1024
  for idx in range(0, y.shape[0] - y.shape[0] % batch_size, batch_size):
    x_batch = x[idx:idx + batch_size]
    y_batch = y[idx:idx + batch_size]
    grads = grad(loss, argnums=(0,1,2))(params1, params2, params3, x_batch, y_batch, activation_indices1, activation_indices2, activation_indices3)
    params1, opt_state1 = adam_update(params1, grads[0], opt_state1, t, lr)
    params2, opt_state2 = adam_update(params2, grads[1], opt_state2, t, lr)
    params3, opt_state3 = adam_update(params3, grads[2], opt_state3, t, lr)
  return params1, params2, params3, opt_state1, opt_state2, opt_state3


# Example usage
layer_sizes1 = [2*768,  768]  # Depth and width of the first network
layer_sizes2 = [768,  768]   # Depth and width of the second network
layer_sizes3 = [768,  768]   # Depth and width of the thrid network
activation_indices1 = [2]  # Activation functions for each layer of the first network: 0=ReLU, 1=Sigmoid, 2=Tanh, 3=Softmax
activation_indices2 = [2]  # Activation functions for each layer of the second network
activation_indices3 = [2]  # Activation functions for each layer of the thrid network

key = random.PRNGKey(0)
params1 = init_network_params(layer_sizes1, key)
key = random.PRNGKey(0)
params2 = init_network_params(layer_sizes2, key)
key = random.PRNGKey(0)
params3 = init_network_params(layer_sizes3, key)

opt_state1 = init_adam_state(params1)
opt_state2 = init_adam_state(params2)
opt_state3 = init_adam_state(params3)

store = loss(params1, params2, params3, x, y, activation_indices1, activation_indices2, activation_indices3)/(x.shape[0]*768)
print(jnp.mean(jnp.abs(y)))
print(store)
# Training loop
learning_rate = 0.0005
for _ in range(1):
    for epoch in range(50):
        t = epoch + 1
        params1, params2, params3, opt_state1, opt_state2, opt_state3 = update(params1, params2, params3, opt_state1, opt_state2, opt_state3,
                                                                               x, y,
                                                                               activation_indices1, activation_indices2, activation_indices3,
                                                                               t, learning_rate)
        if epoch % 10 == 0:
            print(epoch,loss(params1, params2, params3, x, y, activation_indices1, activation_indices2, activation_indices3)/(x.shape[0]*768),loss(params1, params2, params3, x_test, y_test, activation_indices1, activation_indices2, activation_indices3)/(x_test.shape[0]*768))
    print()
    print(store,loss(params1, params2, params3, x, y, activation_indices1, activation_indices2, activation_indices3)/(x.shape[0]*768))

0.08802685
0.08849631
0 0.06639786 0.06502562
10 0.061860185 0.061552074
20 0.0612274 0.061312605
30 0.060859025 0.06126066
40 0.060546182 0.06120682

0.08849631 0.06031722


In [7]:
import pickle
with open('params1_4', 'wb') as f:
    pickle.dump(params1, f)
with open('params2_4', 'wb') as f:
    pickle.dump(params2, f)

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LayerNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
perdector=Sequential()
perdector.add(Dense(50257 , input_shape=(768,), activation='softmax'))
perdector.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use categorical crossentropy for multi-class classification
              metrics=['accuracy'])

In [9]:
LLM_x_test = process_tokens_vectorized(params1, params2, params3, x_test, activation_indices1, activation_indices2, activation_indices3)
LLM_x_test = np.array(jnp.transpose(LLM_x_test , (1,0,2))[0])
print(LLM_x_test.shape)
LLM_y_test = np.array(LLM_y_test)
LLM_y_test =  LLM_y_test.reshape(-1)

(3692, 768)


In [10]:
print(type(LLM_x_test),type(LLM_y_test))
print(LLM_x_test.shape,LLM_y_test.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(3692, 768) (3692,)


In [15]:
process_tokens_vectorized = jax.vmap(fun = process_tokens_single_example, in_axes=(None, None, None, 0, None, None, None))
LLM_x = process_tokens_vectorized(params1, params2, params3, x, activation_indices1, activation_indices2, activation_indices3)
LLM_x = np.array(jnp.transpose(LLM_x , (1,0,2))[0])
print(LLM_x.shape)
LLM_y = np.array(LLM_y)
LLM_y =  LLM_y.reshape(-1)
print(type(LLM_x),type(LLM_y))
print(LLM_x.shape,LLM_y.shape)
perdector.fit(LLM_x, LLM_y, epochs=20, batch_size=128, verbose=1,validation_data=(LLM_x_test,LLM_y_test))

(41358, 768)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(41358, 768) (41358,)
Epoch 1/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.2427 - loss: 8.8466 - val_accuracy: 0.2535 - val_loss: 5.7766
Epoch 2/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.2721 - loss: 5.4267 - val_accuracy: 0.2703 - val_loss: 5.4839
Epoch 3/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.2907 - loss: 5.0643 - val_accuracy: 0.2765 - val_loss: 5.3796
Epoch 4/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.3002 - loss: 4.8713 - val_accuracy: 0.2803 - val_loss: 5.3160
Epoch 5/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.3086 - loss: 4.7142 - val_accuracy: 0.2830 - val_loss: 5.2753
Epoch 6/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.3141 - loss: 4.5725 - val_accuracy: 0.2852 - val_loss: 5.2470
Epoch 7/20
324/324 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.3215 - loss: 4.4639 - val_accuracy: 0.2860 - val_loss: 5.2253
Epoch 8/20
324

In [16]:
perdector.evaluate(LLM_x_test,LLM_y_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3047 - loss: 5.1629


[5.172879219055176, 0.30281689763069153]

In [17]:
perdector.save("perdector_4.h5")